In [8]:
datos = read.csv("/home/emiliano/EstadisticaAplicada/Estadistica.Aplicada.2018/03_efectos_aleatorios_y_mixtos/datos/Colesterol.csv", sep = ",")
head(datos)
str(datos)
attach(datos)
corridaf = as.factor(corrida)
pacientef = as.factor(paciente)

corrida,paciente,colesterol
1,1,167.3
1,1,166.7
2,1,179.6
2,1,175.3
3,1,169.4
3,1,165.9


'data.frame':	40 obs. of  3 variables:
 $ corrida   : int  1 1 2 2 3 3 4 4 1 1 ...
 $ paciente  : int  1 1 1 1 1 1 1 1 2 2 ...
 $ colesterol: num  167 167 180 175 169 ...


**a)** Escriba un modelo lineal para el experimento, suponiendo que los pacientes y las corridas son efectos aleatorios (por qué?), explique los términos y realice el análisis de varianza.

$$y_{ijk} = \mu + paciente_i + corrida_j +(paciente:corrida)_{ij} + \epsilon_{ijk}$$

donde paciente y corrida son efectos aleatorios. 

- *Paciente:* es aleatorio porque fueron seleccionados al azar y no interesa saber la diferencia entre las medias de un paciente específico a otro, sino que lo importante es analizar la varianza de la distribución del efecto de los pacientes. 
- *Corrida:* es un efecto aleatorio porque ... **ESTUDIAR!!!**

In [11]:
library(lme4)
modelo = lmer(colesterol~1 + (1|pacientef) + (1|corridaf) + (1|pacientef:corridaf))
summary(modelo)
# aca vemos el orden:
# 1) pacientef:corridaf
# 2) pacientef
# 3) corridaf

Linear mixed model fit by REML ['lmerMod']
Formula: 
colesterol ~ 1 + (1 | pacientef) + (1 | corridaf) + (1 | pacientef:corridaf)

REML criterion at convergence: 234.3

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-2.0127 -0.5037  0.1301  0.5603  2.1510 

Random effects:
 Groups             Name        Variance Std.Dev.
 pacientef:corridaf (Intercept)    1.258  1.121  
 pacientef          (Intercept) 1642.864 40.532  
 corridaf           (Intercept)   35.533  5.961  
 Residual                          6.608  2.571  
Number of obs: 40, groups:  pacientef:corridaf, 20; pacientef, 5; corridaf, 4

Fixed effects:
            Estimate Std. Error t value
(Intercept)   167.44      18.38   9.112

In [12]:
# ahora que sabemos el orden vemos los IC
confint(modelo)

Computing profile confidence intervals ...


,2.5 %,97.5 %
.sig01,0.000000,2.885759
.sig02,21.725081,79.271621
.sig03,3.050675,17.113633
.sigma,1.941092,3.533180
(Intercept),128.010427,206.859576


En los IC previos observamos que la interacción de los efectos aleatorios no es significativa que que el límite inferior es 0. 

**b)** Muestre los cuadrados medios esperados para el análisis de varianza.

Para esto vamos a usar la librería ``EMSaov``

In [15]:
library(EMSaov)
# agregamos como columnas los factores
datos$pacientef = pacientef
datos$corridaf = corridaf

m = EMSanova(colesterol~pacientef + corridaf, type = c('R', 'R'), data = datos) 
# esto te da los cuadrados medios esperados
print(m) 

                   Df         SS           MS    Fvalue  Pvalue Sig
pacientef           4 52608.1035 13152.025875 1441.4956 <0.0001 ***
corridaf            3  1093.3710   364.457000   39.9454 <0.0001 ***
pacientef:corridaf 12   109.4865     9.123875    1.3806   0.253    
Residuals          20   132.1700     6.608500                      
                                                    EMS
pacientef          Error+2pacientef:corridaf+8pacientef
corridaf           Error+2pacientef:corridaf+10corridaf
pacientef:corridaf            Error+2pacientef:corridaf
Residuals                                         Error


In [17]:
# Cuadrados medios esperados calculados siguiendo la tabla previa
(Epaciente = 6.608500 + 2*9.123875 + 8*13152.025875)
(Ecorrida = 6.608500 + 2*9.123875 + 10*364.457000)
(Einteraccion = 6.608500 + 2*9.123875)

[1] 105241.1

[1] 3669.426

[1] 24.85625

**c)** Estime las componentes de la varianza para corridas, pacientes e interacción.

PREGUNTAR PORQUE NO SABEMOS COMO OBTENER LOS VALORES

**d)** Establezca las hipótesis nula y alternativa para los efectos principales y de interacción, pruebe cada una de las hipótesis nulas e interprete sus resultados.

**SABIENDO DE LOS CONFINT QUE LA INTERACCION NO ES SIGNIFICATIVA, ¿¿DEBEMOS HACER LA COMPARACIÓN TAMBIÉN ENTRE LOS EFECTOS PRINCIPALES O NO HACE FALTA??**

No, no hace falta hacer ambas en efectos aleatorios, si usamos máxima verosimilitud miramos solamente los IC. Pero, para efectos fijos sí es conveniente hacer el test.


Hipótesis para la interacción:

- H0: $\sigma^2_{interaccion} = 0 $
- H1: $\sigma^2_{interaccion} > 0 $

Para esto comparamos el modelo completo contra uno reducido que excluya el término de interacción

In [18]:
# modelo reducido sin la interaccion
modelo2 = lmer(colesterol~1 + (1|pacientef) + (1|corridaf))
anova(modelo, modelo2)

refitting model(s) with ML (instead of REML)


,Df,AIC,BIC,logLik,deviance,Chisq,Chi Df,Pr(>Chisq)
modelo2,4,250.2463,257.0018,-121.1232,242.2463,NA,NA,NA
modelo,5,251.8470,260.2914,-120.9235,241.8470,0.3992791,1,0.5274618


El pvalor nos da grande aceptamos la H0, que nos dice que la interacción no es significativa. 
Como el modelo es **puramente de efectos aleatorios**, es resultado debe ser equivalente a usar el IC que nos devolvió el **confint**.